<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Github_Repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
from IPython.display import clear_output
%pip install GitPython
clear_output()

import git, os, pathlib, sys
from os.path import *

from google.colab import drive, files
drive.mount("/gdrive", force_remount=True)
%cd "/gdrive/My Drive"
%pwd

from BashColors import C
import FileUtilities
from FileUtilities import *
fu.listFiles(silent=True)

In [ ]:
%%writefile GithubRepos.py
# 09/06/2020
class GithubRepos(object):
    import os
    gr = None
    __all__ = sorted(['getCoLabRepo', 'getCurrentProjectRepo', 'gr',
                      'listRemotes', 'repoList'])
    def __dir__(self):
        self.repoList:list
        
    def __init__(self):
        self.gr = GithubRepos
        self.repoList = []
        
    def getCurrentProjectRepo(self):
        currentProjectURL = 'https://github.com/iypc-team/CurrentProject'
        
        localURL = join(fu.cwdPath, basename(currentProjectURL))
        print(basename(currentProjectURL))
        if not exists(localURL):
            currentProjectRepo = git.Repo.clone_from(currentProjectURL, localURL)
            self.repoList.append(currentProjectRepo)
        else: print(f'{C.HEADER}{basename(localURL)} already exists')
    
    def getCoLabRepo(self):
        thisRepo = None
        colabURL = 'https://github.com/iypc-team/CoLab'
        localURL = join(fu.cwdPath, basename(colabURL))
        if not exists(localURL):
            thisRepo = git.Repo.clone_from(colabURL, localURL)
            self.repoList.append(thisRepo)
        else: print(f'{C.GREEN}{thisRepo}')

    def listRemotes(self):
        print(f'{C.WHITE}Remotes:')
        for rpo in self.repoList:
            print(rpo)
            print(f'{rpo.name} {rpo.url}')

    def updateRepoList(self, name):
        if not name in self.repoList:
            self.repoList.append(name)
            
gr = GithubRepos()

In [ ]:
import GithubRepos
from GithubRepos import *

repoName = 'https://github.com/iypc-team'


gr.updateRepoList(name=repoName)


In [ ]:
gr.repoList